# Universidade Federal de Minas Gerais
## Teoria da Decisão - ELE088 
Daniela Amaral Sampaio - 2017074351

Matheus Brito Faria - 2017074386

Victor Emannuel - 2017074394

Importando as bibliotecas que serão utilizadas

In [1]:
import numpy as np
from sklearn import preprocessing
import pandas as pd

Fazendo a importação do drive com as tabelas para o trabalho

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Obtendo os dados do arquivo i5x25.xlsx disponível para o trabalho computacional

O método Analytic Hierarchy Process (AHP) se caracteriza por uma teoria de medição baseada em comparação par a par das alternativas. Ele sustenta-se sobre uma escala de prioridades definida a partir de julgamentos de  \textit{experts} na área do problema. Os julgamentos podem ser inconsistentes, mas o AHP permite tanto quantificar essa consistência quanto melhorar tais julgamentos.

In [3]:
path = r'/content/drive/MyDrive/UFMG/9º SEMESTRE/TEORIA DA DECISAO/' \
        'Trabalho-Computacional/i5x25.xlsx'
data = pd.read_excel(path, header=1)
time_job_machine = data[[1, 2, 3, 4, 5]][:25].to_numpy()
cost_late_job = data.iloc[:25, -1].to_numpy()
due_date = data.iloc[-1, 1]

Inicialmente, nesse método, o processo é de definir o problema e determinar o tipo de conhecimento requerido. Após isso, estrutura-se a hierarquia de decisão de cima para baixo, especificando o objetivo principal, os critérios e as alternativas. Logo após, constroe-se um conjunto de matrizes de comparação par a par, considerando os diferentes níveis de hierarquia.

In [4]:
solutions = [
    [
        [18, 24, 3, 20, 14],
        [23, 22, 0, 9, 5, 19],
        [15, 21, 6, 17, 12],
        [1, 11, 16, 8, 10],
        [2, 13, 7, 4],
    ],
    [
        [11, 24, 18, 14, 20],
        [22, 23, 9, 0, 5, 19],
        [15, 7, 6, 17, 12],
        [1, 21, 8, 16, 10],
        [2, 3, 13, 4],
    ],
    [
        [24, 3, 18, 20, 14],
        [22, 9, 23, 0, 5, 19],
        [8, 15, 7, 6, 17],
        [11, 21, 1, 16, 10],
        [13, 4, 2, 12],
    ],
]

In [5]:
class CriterionMaker:
    def __init__(self, 
                 schedule=None,
                 time_job_machine=time_job_machine, 
                 cost_late_job=cost_late_job, 
                 due_date=due_date):
        self.time_job_machine = time_job_machine
        self.cost_late_job = cost_late_job
        self.due_date = due_date
        self.number_of_machines = time_job_machine.shape[1]
        self.number_of_jobs = time_job_machine.shape[0]
        self.schedule = schedule
        self.calculate_time_to_finish()
        
    def calculate_time_to_finish(self):
        time_to_finish = np.zeros(self.number_of_jobs)
        for machine, machine_schedule in enumerate(self.schedule):
            late_time = 0
            for job in machine_schedule:
                time_to_finish[job] = self.time_job_machine[job, machine] + late_time
                late_time = time_to_finish[job]
        self.time_to_finish = time_to_finish 

    def calculate_lateness(self):
        return np.maximum(self.time_to_finish-self.due_date, 
                          np.zeros(self.number_of_jobs))

    def calculate_lateness_cost(self):
        return np.sum(self.calculate_lateness()*self.cost_late_job)

    def calculate_advance(self):
        return np.maximum(self.due_date-self.time_to_finish, 
                          np.zeros(self.number_of_jobs))
        
    def calculate_advance_save(self):
        return np.sum(self.calculate_advance()*self.cost_late_job)

    def calculate_makespan(self):
        return np.max(self.time_to_finish)

    def calculate_advance_lateness_tradeoff(self):
        return np.sum((self.due_date-self.time_to_finish)*self.cost_late_job)

    def calculate_sum_job_done(self):
        return np.sum(self.due_date-self.time_to_finish >= 0)

    def run(self):
        return (
            -self.calculate_makespan(),
            -self.calculate_lateness_cost(),
            self.calculate_advance_save(),
            self.calculate_sum_job_done(),
        )

In [6]:
data = list()
for solution in solutions:
    data.append(CriterionMaker(solution).run())
data = np.array(data)
data

array([[-11., -73., 290.,  15.],
       [-12., -69., 196.,  14.],
       [-13., -66., 245.,  14.]])

In [7]:
def generate_criteria_matrix(data):
    length = len(data)
    result = np.eye(length)
    num_solutions = range(length)
    for i in num_solutions:
        for j in num_solutions:
            result[i][j] = data[i]/data[j]
    return result

In [8]:
makespan = generate_criteria_matrix(data[:, 0])
lateness_cost = generate_criteria_matrix(data[:, 1])
advance_save = generate_criteria_matrix(data[:, 2])
jobs_done = generate_criteria_matrix(data[:, 3])

In [9]:
makespan

array([[1.        , 0.91666667, 0.84615385],
       [1.09090909, 1.        , 0.92307692],
       [1.18181818, 1.08333333, 1.        ]])

Por fim, utiliza-se as prioridades obtidas por meio das comprações em cada nível para gerar a prioridade global das alternativas.

In [11]:
def calculate_priority(data):
    priority = np.empty(data.shape)
    sum_columns = data.sum(axis=0)
    for column in range(data.shape[1]):
        priority[:, column] = data[:, column] / sum_columns[column]
    return priority.mean(axis=1)

In [ ]:
priorities = np.array([
              calculate_priority(makespan),
              calculate_priority(lateness_cost),
              calculate_priority(advance_save),
              calculate_priority(jobs_done),
]).T
priorities

array([[0.30555556, 0.35096154, 0.39671683, 0.34883721],
       [0.33333333, 0.33173077, 0.26812585, 0.3255814 ],
       [0.36111111, 0.31730769, 0.33515732, 0.3255814 ]])

In [12]:
calculate_priority(makespan)

array([0.30555556, 0.33333333, 0.36111111])

In [ ]:
# makespan,
# lateness_cost,
# advance_save,
# jobs_done

weights_criterias = np.array([
                              [1  , 1/7, 1/9, 3 ],
                              [7  , 1  , 1/3, 7 ],
                              [9  , 3  , 1  , 9 ],
                              [1/3, 1/7 , 1/9, 1],
])
(np.linalg.eig(weights_criterias)[0].max() - 4)/3

(0.08191267231290547+0j)

In [ ]:
priority_criterias = calculate_priority(weights_criterias)
priority_criterias

array([0.07811355, 0.3003663 , 0.57802198, 0.04349817])

In [ ]:
scores = list()
for solution in priorities:
    scores.append(solution.dot(priority_criterias))

scores

[0.37376987333777295, 0.2948234263660971, 0.33140670029612995]